[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/pixeltable/pixeltable/blob/release/docs/notebooks/integrations/working-with-openai.ipynb)&nbsp;&nbsp;
[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pixeltable/pixeltable/blob/release/docs/notebooks/integrations/working-with-openai.ipynb)&nbsp;&nbsp;
<a href="https://raw.githubusercontent.com/pixeltable/pixeltable/release/docs/notebooks/integrations/working-with-openai.ipynb" download><img src="https://img.shields.io/badge/%E2%AC%87-Download%20Notebook-blue" alt="Download Notebook"></a>

# Working with OpenAI in Pixeltable

Pixeltable's OpenAI integration enables you to access OpenAI models via the OpenAI API.

### Prerequisites
- An OpenAI account with an API key (https://openai.com/index/openai-api/)

### Important Notes

- OpenAI usage may incur costs based on your OpenAI plan.
- Be mindful of sensitive data and consider security measures when integrating with external services.

First you'll need to install required libraries and enter your OpenAI API key.

In [ ]:
%pip install -qU pixeltable openai

In [ ]:
import os
import getpass
if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter your OpenAI API key:')

Now let's create a Pixeltable directory to hold the tables for our demo.

In [1]:
import pixeltable as pxt

# Remove the 'openai_demo' directory and its contents, if it exists
pxt.drop_dir('openai_demo', force=True)
pxt.create_dir('openai_demo')

Connected to Pixeltable database at: postgresql+psycopg://postgres:@/pixeltable?host=/Users/asiegel/.pixeltable/pgdata
Created directory 'openai_demo'.


## Chat Completions

Create a Table: In Pixeltable, create a table with columns to represent your input data and the columns where you want to store the results from OpenAI.

In [2]:
from pixeltable.functions import openai

# Create a table in Pixeltable and add a computed column that calls OpenAI

t = pxt.create_table('openai_demo.chat', {'input': pxt.String})

messages = [{'role': 'user', 'content': t.input}]
t.add_computed_column(output=openai.chat_completions(
    messages=messages,
    model='gpt-4o-mini',
    model_kwargs={
        # Optional dict with parameters for the OpenAI API
        'max_tokens': 300,
        'top_p': 0.9,
        'temperature': 0.7
    }
))

Created table `chat`.
Added 0 column values with 0 errors.


UpdateStatus(num_rows=0, num_computed_values=0, num_excs=0, updated_cols=[], cols_with_excs=[])

In [3]:
# Parse the response into a new column
t.add_computed_column(response=t.output.choices[0].message.content)

Added 0 column values with 0 errors.


UpdateStatus(num_rows=0, num_computed_values=0, num_excs=0, updated_cols=[], cols_with_excs=[])

In [4]:
# Start a conversation
t.insert(input="How many islands are in the Aleutian island chain?")
t.select(t.input, t.response).head()

Inserting rows into `chat`: 1 rows [00:00, 106.84 rows/s]
Inserted 1 row with 0 errors.


input  \
0  How many islands are in the Aleutian island ch...   

                                            response  
0  The Aleutian Islands are a chain of over 300 i...

## Embeddings

In [5]:
emb_t = pxt.create_table('openai_demo.embeddings', {'input': pxt.String})
emb_t.add_computed_column(embedding=openai.embeddings(
    input=emb_t.input,
    model='text-embedding-3-small'
))

Created table `embeddings`.
Added 0 column values with 0 errors.


UpdateStatus(num_rows=0, num_computed_values=0, num_excs=0, updated_cols=[], cols_with_excs=[])

In [6]:
emb_t.insert(input='OpenAI provides a variety of embeddings models.')

Inserting rows into `embeddings`: 1 rows [00:00, 164.18 rows/s]
Inserted 1 row with 0 errors.


UpdateStatus(num_rows=1, num_computed_values=2, num_excs=0, updated_cols=[], cols_with_excs=[])

In [7]:
emb_t.head()

input  \
0  OpenAI provides a variety of embeddings models.   

                                           embedding  
0  [-0.022908814, -0.044679735, 0.06892383, -0.01...

## Image Generations

In [8]:
image_t = pxt.create_table('openai_demo.images', {'input': pxt.String})
image_t.add_computed_column(img=openai.image_generations(
    image_t.input,
    model='dall-e-2',
))

Created table `images`.
Added 0 column values with 0 errors.


UpdateStatus(num_rows=0, num_computed_values=0, num_excs=0, updated_cols=[], cols_with_excs=[])

In [9]:
image_t.insert([
    {'input': 'A giant Pixel floating in the open ocean in a sea of data'}
])

Inserting rows into `images`: 1 rows [00:00, 444.88 rows/s]
Inserted 1 row with 0 errors.


UpdateStatus(num_rows=1, num_computed_values=2, num_excs=0, updated_cols=[], cols_with_excs=[])

In [10]:
image_t

Table 'openai_demo.images'

 Column Name                 Type                               Computed With
       input               String                                            
         img  Image[(1024, 1024)]  image_generations(input, model='dall-e-2')

In [11]:
image_t.head()

input  \
0  A giant Pixel floating in the open ocean in a ...   

                                                 img  
0  <PIL.JpegImagePlugin.JpegImageFile image mode=...

## Audio Transcription

In [13]:
audio_t = pxt.create_table('openai_demo.audio', {'input': pxt.Audio})
audio_t.add_computed_column(result=openai.transcriptions(
    audio_t.input,
    model='whisper-1',
    model_kwargs={
        'language': 'en',
        'prompt': 'Transcribe the contents of this recording.'
    },
))

Created table `audio`.
Added 0 column values with 0 errors.


UpdateStatus(num_rows=0, num_computed_values=0, num_excs=0, updated_cols=[], cols_with_excs=[])

In [14]:
url = (
    'https://github.com/pixeltable/pixeltable/raw/release/tests/data/audio/'
    'jfk_1961_0109_cityuponahill-excerpt.flac'
)
audio_t.insert([{'input': url}])

Inserting rows into `audio`: 1 rows [00:00, 160.69 rows/s]
Inserted 1 row with 0 errors.


UpdateStatus(num_rows=1, num_computed_values=3, num_excs=0, updated_cols=[], cols_with_excs=[])

In [15]:
audio_t.head()

input  \
0  /Users/asiegel/.pixeltable/file_cache/a3235626...   

                                              result  
0  {'text': 'Allow me to illustrate. During the l...

In [16]:
audio_t.head()[0]['result']['text']

'Allow me to illustrate. During the last 60 days, I have been at the task of constructing an administration. It has been a long and deliberate process. Some have counseled greater speed. Others have counseled more expedient tests. But I have been guided by the standard John Winthrop set before his shipmates on the flagship Arabella 331 years ago, as they too faced the task of building a new government on a perilous frontier. We must always consider, he said, that we shall be as a city upon a hill. The eyes of all peoples are upon us. Today, the eyes of all people are truly upon us. And our governments, in every branch, at every level,'

### Learn More

To learn more about advanced techniques like RAG operations in Pixeltable, check out the [RAG Operations in Pixeltable](https://pixeltable.readme.io/docs/rag-operations-in-pixeltable) tutorial.

If you have any questions, don't hesitate to reach out.